<a href="https://colab.research.google.com/github/fayrouz-mikhael/DataScience-Project/blob/master/project_Using3modelscomparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor 
from google.colab import files
%matplotlib inline


In [0]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)

In [193]:
train_df.sample(5).transpose()

rental_id,7139746,7373257,7341304,7368613,7259923
addr_unit,#3,#1B,#1C,#2,#2C
building_id,409330,130468,84388,409078,25639
bedrooms,4,3,2,4,1
bathrooms,1,2,2,1,1
size_sqft,950,1200,0,1200,0
created_at,2018-05-02 17:25:27,2018-07-17 09:14:52,2018-07-06 19:46:41,2018-07-15 18:27:29,2018-06-11 11:02:34
addr_street,192 CORNELIA STREET,44 BENNETT AVENUE,238 EAST 82 STREET,1117 JEFFERSON AVENUE,29 WEST 12 STREET
addr_city,Brooklyn,New York,New York,Brooklyn,New York
addr_zip,11221,10033,10028,11221,10011
addr_lat,40.6921,40.852,40.7754,40.6893,40.7353


In [0]:
submit1_df['min_to_subway'] = submit1_df['min_to_subway'].fillna(submit1_df['min_to_subway'].mean())
submit1_df.year_built = submit1_df.year_built.fillna(submit1_df.year_built.median(), axis=0)


In [0]:
# extract usable features
feature_cols = [
    'bedrooms', 'year_built', 'bathrooms'
]
train_features = train_df[feature_cols] 

# impute missing values with medians
train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
train_target = train_df['rent']

**Fit Model**

In [0]:
lreg = LinearRegression()
lreg.fit(train_features, train_target)

rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
rf.fit(train_features, train_target);

DTR = DecisionTreeRegressor(random_state = 0)
DTR.fit(train_features, train_target);

In [216]:
test_features = test_df[feature_cols] 

# impute missing values with medians
test_features = test_features.fillna(train_features.median(), axis=0)

# construct predictions 
test_df['predicted'] = DTR.predict(test_features)

mean_squared_error(test_df['rent'], test_df['predicted'])

5052622.901633972

In [217]:
master_df = train_df.append(test_df, sort=False)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']

DTR.fit(master_features, master_target)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=0, splitter='best')

**LREG:Create Submission File for test2**

In [230]:
submit1_features = submit1_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit1_df['predictions'] = lreg.predict(submit1_features)
submit1_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit1_df['fake_rent'] = np.ones(submit1_df['predictions'].shape) * master_target.median()
mean_squared_error(submit1_df['predictions'], submit1_df['fake_rent'])


3098061.112076225

In [0]:
outputdf = submit1_df[['predictions']]
outputdf.to_csv('output.csv')
files.download('output.csv')

**RF:Create Submission File for test2**

In [218]:
submit1_featuresRF = test_df[feature_cols].fillna(train_features.median(), axis=0)
submit1_df['predictions'] = rf.predict(submit1_featuresRF)
submit1_df['predictions'].to_csv('sample_submission1.csv', header=True)
submit1_df['fake_rent'] = np.ones(submit1_df['predictions'].shape) * master_target.median()

print('Mean squared Error:', mean_squared_error( submit1_df['predictions'],submit1_df['fake_rent']))

Mean squared Error: 5321298.255968601


**DT:Create Submission File for test2**

In [220]:
submit1_featuresDTR = test_df[feature_cols].fillna(train_features.median(), axis=0)
submit1_df['predictions'] = DTR.predict(submit1_featuresDTR)

submit1_df['predictions'].to_csv('sample_submission1.csv', header=True)
submit1_df['fake_rent'] = np.ones(submit1_df['predictions'].shape) * master_target.median()

print('Mean squared Error:', mean_squared_error(submit1_df['predictions'],submit1_df['fake_rent']))

Mean squared Error: 5884941.237391475


In [202]:
submit1_df.isna().sum()


addr_unit                   29
building_id                  0
bedrooms                     0
bathrooms                    0
size_sqft                    0
created_at                   0
addr_street                  0
addr_city                    0
addr_zip                     0
addr_lat                     0
addr_lon                     0
bin                          0
bbl                          0
floor_count                  0
year_built                   0
min_to_subway                0
has_doorman                  0
has_elevator                 0
has_fireplace                0
has_dishwasher               0
is_furnished                 0
has_gym                      0
allows_pets                  0
has_washer_dryer             0
has_garage                   0
has_roofdeck                 0
has_concierge                0
has_pool                     0
has_garden                   0
has_childrens_playroom       0
rent                      2000
no_fee                       0
descript